In [3]:
# I installed the needed libraries only for my user account.
# This avoids the "Permission denied" error.

!pip install --user google-api-python-client python-dotenv pandas openpyxl nltk


  Using cached google_api_python_client-2.187.0-py3-none-any.whl.metadata (7.0 kB)
  Using cached openpyxl-3.1.5-py2.py3-none-any.whl.metadata (2.5 kB)
  Using cached google_auth-2.43.0-py2.py3-none-any.whl.metadata (6.6 kB)
  Using cached google_auth_httplib2-0.2.1-py3-none-any.whl.metadata (3.0 kB)
  Using cached google_api_core-2.28.1-py3-none-any.whl.metadata (3.3 kB)
  Using cached et_xmlfile-2.0.0-py3-none-any.whl.metadata (2.7 kB)
  Using cached googleapis_common_protos-1.72.0-py3-none-any.whl.metadata (9.4 kB)
Using cached google_api_python_client-2.187.0-py3-none-any.whl (14.6 MB)
Using cached openpyxl-3.1.5-py2.py3-none-any.whl (250 kB)
Using cached google_api_core-2.28.1-py3-none-any.whl (173 kB)
Using cached google_auth-2.43.0-py2.py3-none-any.whl (223 kB)
Using cached google_auth_httplib2-0.2.1-py3-none-any.whl (9.5 kB)
Using cached et_xmlfile-2.0.0-py3-none-any.whl (18 kB)
Using cached googleapis_common_protos-1.72.0-py3-none-any.whl (297 kB)


In [25]:
# ============================================================
# IST652 Mini Project – F1 YouTube Comments + Kaggle Race Data
# ============================================================

# Installing packages

import os
import re
import time
from collections import Counter

import pandas as pd
import numpy as np

from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from dotenv import load_dotenv

import nltk
from nltk.sentiment import SentimentIntensityAnalyzer

In [ ]:

# --------------------------
# 0. Setup: NLTK + API key
# --------------------------

load_dotenv()
nltk.download("vader_lexicon", quiet=True)
sia = SentimentIntensityAnalyzer()

# YouTube API key:
YOUTUBE_API_KEY = "YOUTUBE_API_KEY"

if not YOUTUBE_API_KEY or YOUTUBE_API_KEY == "PASTE_YOUR_REAL_API_KEY_HERE":
    raise ValueError("Please paste your actual YouTube API key in YOUTUBE_API_KEY.")

youtube = build("youtube", "v3", developerKey=YOUTUBE_API_KEY)


In [28]:
# =====================================
# 1. ting data from YouTube (API source)
# =====================================

F1_CHANNEL_ID = "UCB_qr75-ydFVKSF9Dmo6izg"  # Formula 1 official channel


def get_race_highlight_videos(youtube_client, max_videos=12):
    """
    Search the Formula 1 channel for 'Race Highlights' videos
    and return a dataframe with videoId, title, and publishedAt.
    """
    videos = []
    next_page_token = None

    while len(videos) < max_videos:
        request = youtube_client.search().list(
            part="snippet",
            channelId=F1_CHANNEL_ID,
            q="Race Highlights",
            type="video",
            order="date",
            maxResults=min(50, max_videos - len(videos)),
            pageToken=next_page_token,
        )
        response = request.execute()

        for item in response.get("items", []):
            videos.append(
                {
                    "videoId": item["id"]["videoId"],
                    "title": item["snippet"]["title"],
                    "publishedAt": item["snippet"]["publishedAt"],
                }
            )

        next_page_token = response.get("nextPageToken")
        if not next_page_token:
            break

    return pd.DataFrame(videos)


def get_video_stats(youtube_client, video_ids):
    """
    For a list of video IDs, fetch statistics and basic info:
    views, likes, comments, and duration.
    """
    all_stats = []

    for i in range(0, len(video_ids), 50):  # API allows up to 50 IDs per call
        chunk = video_ids[i : i + 50]
        request = youtube_client.videos().list(
            part="snippet,contentDetails,statistics", id=",".join(chunk)
        )
        response = request.execute()

        for item in response.get("items", []):
            stats = item.get("statistics", {})
            snippet = item.get("snippet", {})
            content = item.get("contentDetails", {})

            all_stats.append(
                {
                    "videoId": item["id"],
                    "title": snippet.get("title"),
                    "publishedAt": snippet.get("publishedAt"),
                    "duration": content.get("duration"),
                    "viewCount": int(stats.get("viewCount", 0)),
                    "likeCount": int(stats.get("likeCount", 0)),
                    "commentCount": int(stats.get("commentCount", 0)),
                }
            )

    return pd.DataFrame(all_stats)


def get_comments_for_video(
    youtube_client, video_id, max_comments=200, sleep_seconds=0.2
):
    """
    Fetch up to max_comments top-level comments for one video.
    If comments are disabled, skip the video and return an empty list.
    """
    comments = []
    next_page_token = None

    while len(comments) < max_comments:
        try:
            request = youtube_client.commentThreads().list(
                part="snippet",
                videoId=video_id,
                maxResults=min(100, max_comments - len(comments)),
                textFormat="plainText",
                pageToken=next_page_token,
            )
            response = request.execute()
        except HttpError as e:
            # Comments disabled for this video
            if e.resp.status == 403 and "commentsDisabled" in str(e):
                print(f"Comments are disabled for video {video_id}. Skipping.")
                return []
            else:
                raise

        for item in response.get("items", []):
            top_comment = item["snippet"]["topLevelComment"]["snippet"]
            comments.append(
                {
                    "videoId": video_id,
                    "commentId": item["snippet"]["topLevelComment"]["id"],
                    "author": top_comment.get("authorDisplayName"),
                    "text": top_comment.get("textDisplay"),
                    "likeCount": int(top_comment.get("likeCount", 0)),
                    "publishedAt": top_comment.get("publishedAt"),
                }
            )

        next_page_token = response.get("nextPageToken")
        if not next_page_token:
            break

        time.sleep(sleep_seconds)

    return comments


def get_comments_for_videos(youtube_client, video_ids, max_comments_per_video=150):
    """
    Loop through each video ID and collect comments into a single dataframe.
    """
    all_comments = []
    for vid in video_ids:
        print(f"Fetching comments for video: {vid}")
        video_comments = get_comments_for_video(
            youtube_client, vid, max_comments=max_comments_per_video
        )
        all_comments.extend(video_comments)
    return pd.DataFrame(all_comments)


# --- 1A. Getting videos and stats from YouTube ---

videos_basic_df = get_race_highlight_videos(youtube, max_videos=12)
videos_df = get_video_stats(youtube, videos_basic_df["videoId"].tolist())

# Only keeping videos that have comments
videos_with_comments_df = videos_df[videos_df["commentCount"] > 0].copy()
video_ids_for_comments = videos_with_comments_df["videoId"].tolist()

# --- 1B. Getting comments for those videos ---

comments_df = get_comments_for_videos(
    youtube, video_ids_for_comments, max_comments_per_video=150
)



Fetching comments for video: Jt3gVEtfy1c
Fetching comments for video: 2RKcJhnhtlU
Fetching comments for video: Cb9Wo7sPGVg
Fetching comments for video: 1E1-_beyrNQ
Fetching comments for video: pcmZp9kMVDg
Fetching comments for video: cqJBQ7bx1L4
Fetching comments for video: -o7fAgMnOZA
Fetching comments for video: bijI5emoGgg
Fetching comments for video: MK83clSv6-k
Fetching comments for video: oavaWsg56d4
Fetching comments for video: Sy8Sa9_2yXw
Fetching comments for video: WgBuHqqE7Mw
Fetching comments for video: VJuKqOQBDhs
Fetching comments for video: eeS1_6UF1CI
Fetching comments for video: 4Z3y6mYDJgM
Fetching comments for video: 683MegAx5F4
Fetching comments for video: MYpwaMHYDfw
Fetching comments for video: 0JIQVXsSzps
Fetching comments for video: ZYwWyNnLKa4
Fetching comments for video: 55_LalL3E6Y
Fetching comments for video: -WoFoUPDCow
Fetching comments for video: -dCLabtcZBQ
Fetching comments for video: lWY2naac5f4
Fetching comments for video: iz4TxOSy7Rs


In [29]:
# ===============================================
# 2. Getting data from Kaggle (CSV / second source)
# ===============================================

# Race_Schedule.csv comes from Kaggle.
race_schedule_df = pd.read_csv("Race_Schedule.csv")


In [30]:
# ============================================
# 3. Combining YouTube + Kaggle race schedule
# ============================================

def extract_year_and_race(title):
    """
    From a title like '2024 Bahrain Grand Prix | Race Highlights',
    extract race_year = 2024 and race_name = 'Bahrain Grand Prix'.
    """
    if not isinstance(title, str):
        return None, None

    year_match = re.search(r"(20\d{2})", title)
    year = int(year_match.group(1)) if year_match else None

    gp_match = re.search(r"([A-Za-z\s]+Grand Prix)", title)
    race_name = gp_match.group(1).strip() if gp_match else None

    return year, race_name


def normalize_name(name):
    """
    Normalize race names:
    - cast to string
    - lowercase
    - collapse extra spaces
    """
    if pd.isna(name):
        return None
    name = str(name)
    name = name.lower()
    name = re.sub(r"\s+", " ", name).strip()
    return name


# --- 3A. Extracting year and race name from video titles ---

videos_df[["race_year", "race_name_raw"]] = videos_df["title"].apply(
    lambda t: pd.Series(extract_year_and_race(t))
)
videos_df["race_name_norm"] = videos_df["race_name_raw"].apply(normalize_name)

# --- 3B. Preparing race schedule columns for join ---

cols_lower = [c.lower() for c in race_schedule_df.columns]

if "season" in cols_lower:
    year_col = race_schedule_df.columns[cols_lower.index("season")]
elif "year" in cols_lower:
    year_col = race_schedule_df.columns[cols_lower.index("year")]
else:
    year_col = race_schedule_df.columns[0]

race_name_col = None
for col in race_schedule_df.columns:
    if "grand" in col.lower() or "race" in col.lower() or "gp" in col.lower():
        race_name_col = col
        break
if race_name_col is None:
    race_name_col = race_schedule_df.columns[1]

race_schedule_df["race_year"] = race_schedule_df[year_col]
race_schedule_df["race_name_raw"] = race_schedule_df[race_name_col]
race_schedule_df["race_name_norm"] = race_schedule_df["race_name_raw"].apply(
    normalize_name
)

race_schedule_join = race_schedule_df[["race_year", "race_name_norm"]].drop_duplicates()

# --- 3C. Joining YouTube videos with race schedule ---

videos_merged_df = pd.merge(
    videos_df,
    race_schedule_join,
    on=["race_year", "race_name_norm"],
    how="left",
)

# Also attaching race info to comments via videoId
comments_combined_df = comments_df.merge(
    videos_merged_df[["videoId", "race_year", "race_name_norm"]],
    on="videoId",
    how="left",
)


In [32]:
# ==============================
# 4. Analysis + Output files
# ==============================

# ---- 4.1 Analysis 1: Video-level engagement summary ----

q1_df = videos_merged_df.copy()
q1_df["like_view_ratio"] = q1_df["likeCount"] / q1_df["viewCount"].replace(0, np.nan)
q1_df["comment_view_ratio"] = q1_df["commentCount"] / q1_df["viewCount"].replace(
    0, np.nan
)

q1_df_out = q1_df[
    [
        "videoId",
        "title",
        "race_year",
        "race_name_raw",
        "viewCount",
        "likeCount",
        "commentCount",
        "like_view_ratio",
        "comment_view_ratio",
    ]
]

q1_df_out.to_csv("output_f1_video_summary.csv", index=False)


# ---- 4.2 Adding driver mentions + sentiment to comments ----

drivers_list = [
    "Verstappen",
    "Hamilton",
    "Russell",
    "Norris",
    "Leclerc",
    "Sainz",
    "Perez",
    "Piastri",
    "Alonso",
    "Tsunoda",
    "Stroll",
    "Gasly",
    "Ocon",
    "Hulkenberg",
    "Bottas",
    "Zhou",
    "Ricciardo",
    "Magnussen",
]


def find_drivers_in_text(text, drivers):
    if pd.isna(text):
        return []
    found = []
    for d in drivers:
        if re.search(r"\b" + re.escape(d) + r"\b", str(text), flags=re.IGNORECASE):
            found.append(d)
    return list(set(found))


def get_sentiment_label(text):
    if pd.isna(text) or not isinstance(text, str):
        return 0.0, "neutral"
    scores = sia.polarity_scores(text)
    comp = scores["compound"]
    if comp >= 0.05:
        label = "positive"
    elif comp <= -0.05:
        label = "negative"
    else:
        label = "neutral"
    return comp, label


comments_combined_df["drivers_mentioned"] = comments_combined_df["text"].apply(
    lambda t: find_drivers_in_text(t, drivers_list)
)
comments_combined_df[["sentiment_score", "sentiment_label"]] = comments_combined_df[
    "text"
].apply(lambda t: pd.Series(get_sentiment_label(t)))


# ---- 4.3 Analysis 2: Driver mentions & comment likes ----

comments_exploded = comments_combined_df.explode("drivers_mentioned")
comments_exploded = comments_exploded[
    ~comments_exploded["drivers_mentioned"].isna()
]

driver_stats = (
    comments_exploded.groupby("drivers_mentioned")
    .agg(
        total_mentions=("commentId", "count"),
        total_comment_likes=("likeCount", "sum"),
        avg_comment_likes=("likeCount", "mean"),
    )
    .reset_index()
    .rename(columns={"drivers_mentioned": "driver_name"})
)

driver_stats = driver_stats.sort_values("total_mentions", ascending=False)
driver_stats.to_csv("output_f1_driver_mentions.csv", index=False)


# ---- 4.4 Analysis 3: Driver sentiment vs video engagement ----

comments_with_video = comments_exploded.merge(
    videos_merged_df[["videoId", "viewCount", "likeCount", "commentCount"]],
    on="videoId",
    how="left",
    suffixes=("_comment", "_video"),
)

comments_with_video["like_view_ratio"] = comments_with_video["likeCount_video"] / (
    comments_with_video["viewCount"].replace(0, np.nan)
)
comments_with_video["comment_view_ratio"] = comments_with_video["commentCount"] / (
    comments_with_video["viewCount"].replace(0, np.nan)
)

driver_sentiment_engagement = (
    comments_with_video.groupby("drivers_mentioned")
    .agg(
        total_mentions=("commentId", "count"),
        positive_comments=("sentiment_label", lambda x: (x == "positive").sum()),
        negative_comments=("sentiment_label", lambda x: (x == "negative").sum()),
        neutral_comments=("sentiment_label", lambda x: (x == "neutral").sum()),
        avg_comment_sentiment=("sentiment_score", "mean"),
        avg_comment_likes=("likeCount_comment", "mean"),
        avg_video_like_view_ratio=("like_view_ratio", "mean"),
        avg_video_comment_view_ratio=("comment_view_ratio", "mean"),
    )
    .reset_index()
    .rename(columns={"drivers_mentioned": "driver_name"})
)

driver_sentiment_engagement["positive_ratio"] = (
    driver_sentiment_engagement["positive_comments"]
    / driver_sentiment_engagement["total_mentions"]
)

driver_sentiment_engagement = driver_sentiment_engagement.sort_values(
    "total_mentions", ascending=False
)
driver_sentiment_engagement.to_csv(
    "output_f1_driver_sentiment_engagement.csv", index=False
)

print("Analysis complete. CSV files created:")
print(" - output_f1_video_summary.csv")
print(" - output_f1_driver_mentions.csv")
print(" - output_f1_driver_sentiment_engagement.csv")

Analysis complete. CSV files created:
 - output_f1_video_summary.csv
 - output_f1_driver_mentions.csv
 - output_f1_driver_sentiment_engagement.csv
